In [1]:
import pandas as pd

# 대구 버스정류장 근처 지하철 역 개수 세기

In [2]:
train = pd.read_csv("./대구광역시_도시철도역 출구 인근 버스 정류소 정보_20170124..csv", encoding='cp949')
bus = pd.read_csv("./대구광역시_시내버스 정류소 위치정보_20210630.csv", encoding='cp949')

### 지하철 데이터 전처리

In [3]:
train.head()

,도시철도역_ID,도시철도역 명,도시철도역 출구번호,정류소_ID,정류소 명
0,117,대곡,1,7111000100,대곡역(1번출구)
1,117,대곡,2,7041033900,대곡역(한라하우젠트앞)
2,117,대곡,3,7111000200,대곡역(2번출구)
3,117,대곡,4,7041053200,유천교1
4,117,대곡,4,7041053300,유천교2


In [4]:
print(len(train['도시철도역 명'].unique()))
print(len(train['정류소 명'].unique()))

#중복제거
train.drop_duplicates(subset=['도시철도역 명', '정류소 명'], inplace=True)

84
314


In [6]:
g_train = train.groupby(by='정류소_ID').count()
g_train.describe()

,도시철도역_ID,도시철도역 명,도시철도역 출구번호,정류소 명
count,314.000000,314.000000,314.000000,314.000000
mean,1.015924,1.015924,1.015924,1.015924
std,0.125380,0.125380,0.125380,0.125380
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,2.000000,2.000000,2.000000,2.000000


### 정류소와 인근 지하철역 수 데이터프레임 생성

In [23]:
#merge
train_merge = train[['정류소_ID', '정류소 명']]
train_merge = train_merge.merge(right=g_train['도시철도역 명'], on='정류소_ID')
train_merge = train_merge.rename(columns={'도시철도역 명' : '지하철역 수'})
train_merge.head()

,정류소_ID,정류소 명,지하철역 수
0,7111000100,대곡역(1번출구),1
1,7041033900,대곡역(한라하우젠트앞),1
2,7111000200,대곡역(2번출구),1
3,7041053200,유천교1,1
4,7041053300,유천교2,1


In [24]:
train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 318
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   정류소_ID  319 non-null    int64 
 1   정류소 명   319 non-null    object
 2   지하철역 수  319 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 10.0+ KB


### 버스 데이터 전처리

In [9]:
bus.head()

,정류소아이디,모바일아이디,정류소명,영문명,시도,구군,동,경도,위도,경유노선수,경유노선
0,1570000140,NaN,감문리,NaN,대구광역시,달성군,하빈면,128.424005,35.890744,1,20
1,7001000100,5191.0,대명시장앞,Daemyeong Market,대구광역시,남구,대명3동,128.577130,35.858690,7,"509, 650, 706, 805, 836, 달서4, 순환2-1"
2,7001000200,2164.0,대명시장건너,Daemyeong Market,대구광역시,중구,남산4동,128.577815,35.858640,7,"509, 650, 706, 805, 836, 달서4-1, 순환2"
3,7001000300,2166.0,계명대학교대명캠퍼스앞,Keimyung University Daemyeong Campus,대구광역시,남구,대명3동,128.580848,35.857112,7,"509, 650, 706, 805, 836, 달서4, 순환2-1"
4,7001000400,2163.0,계명대학교대명캠퍼스건너,Keimyung University Daemyeong Campus,대구광역시,중구,남산4동,128.580007,35.857675,7,"509, 650, 706, 805, 836, 달서4-1, 순환2"


In [18]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3248 entries, 0 to 3247
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   정류소아이디  3248 non-null   int64 
 1   정류소명    3248 non-null   object
dtypes: int64(1), object(1)
memory usage: 50.9+ KB


In [20]:
print(bus['정류소아이디'].nunique())
print(bus['정류소명'].nunique())

3248
3248


### 버스데이터랑 지하철 데이터 합치기

In [47]:
bus = bus[['정류소아이디', '정류소명']]

d_train_final = pd.merge(left=bus, right=train_merge, left_on='정류소아이디', right_on='정류소_ID', how='outer')
d_train_final.head()

,정류소아이디,정류소명,정류소_ID,정류소 명,지하철역 수
0,1.570000e+09,감문리,NaN,NaN,NaN
1,7.001000e+09,대명시장앞,NaN,NaN,NaN
2,7.001000e+09,대명시장건너,NaN,NaN,NaN
3,7.001000e+09,계명대학교대명캠퍼스앞,NaN,NaN,NaN
4,7.001000e+09,계명대학교대명캠퍼스건너,NaN,NaN,NaN


In [49]:
d_train_final[d_train_final['정류소아이디'].isna()]

,정류소아이디,정류소명,정류소_ID,정류소 명,지하철역 수
3253,NaN,NaN,7.041023e+09,상인현대맨션앞1,1.0
3254,NaN,NaN,7.001001e+09,현대백화점.동아쇼핑앞,1.0
3255,NaN,NaN,7.001004e+09,국민은행대구지점건너,1.0
3256,NaN,NaN,7.001004e+09,중앙로역(4번출구),1.0
3257,NaN,NaN,7.011034e+09,동호CNG(종점),1.0
3258,NaN,NaN,7.031011e+09,크리스탈호텔앞,1.0
3259,NaN,NaN,7.001013e+09,남산휴먼시아앞,1.0
3260,NaN,NaN,7.121005e+09,정평역1번출구,1.0
3261,NaN,NaN,3.600091e+09,정평역2번출구,1.0
3262,NaN,NaN,7.121005e+09,정평역3번출구,1.0


In [45]:
#겹치는 내용 지우기
d_train_final.drop(['정류소_ID', '정류소 명'], axis=1, inplace=True)

#근처에 지하철이 없는 값은 0으로 채우기
d_train_final = d_train_final.fillna('0')

#서울데이터와 열이름 맞추기
d_train_final.rename({'정류소아이디' : '정류소ID'}, axis=1, inplace=True)

d_train_final.head()


,정류소ID,정류소명,지하철역 수
0,1570000140,감문리,0
1,7001000100,대명시장앞,0
2,7001000200,대명시장건너,0
3,7001000300,계명대학교대명캠퍼스앞,0
4,7001000400,계명대학교대명캠퍼스건너,0


In [46]:
d_train_final.to_csv('대구 지하철역수.csv', index=False, encoding='cp949')